# workflow

currently cf simplification_api_knupu notebook in simplification

* install sgeop bleeding edge
* import all raw edges
* to proj crs
* buffer around all edges (custom, now: 20m)
* union of buffer polygons
* get boundary and explode it
* drop too-short linestrings (loops within)
* polygonize
* only keep those polygons that have interiors
* now we have one polygon per network compoentn
* for each polygon:
	* get delineation (all interiors and the exterior)
	* unzip delineation lines, with modulo caveat not to lose geoms
	* skeletonize (delineation lines and comp poly) with sgeop.geometry.voronoi_skeleton
	* convert to nx
	* should be only 1 component!! (#TODO : consolidate nodes with sgeop.nodes.consolidate_nodes OR shapely.snap?) 
	* iteratively remove degree-one nodes if dangling edge is short enough
	* remove false nodes
	* convert back to nx
* combine into one networkx object
* save 

In [ ]:
import geopandas as gpd
import folium
import momepy
import matplotlib.pyplot as plt
import shapely
import numpy as np
from esda.shape import convex_hull_ratio

In [ ]:
proj_crs = 'EPSG:25832'
length_threshold = 750 # in meters

In [ ]:
nodes = gpd.read_file("./data/cykelknudepunkter.geojson").to_crs(proj_crs)
edges = gpd.read_file("./data/cykelknudepunktsstraekninger.geojson").to_crs(proj_crs)

In [ ]:
nodes.head(3)

In [ ]:
edges.head(3)

In [ ]:
geom = edges.buffer(20).union_all()
ser = gpd.GeoSeries([geom], crs = edges.crs)
bou = ser.boundary
bou = bou.explode().reset_index(drop=True)
bou = gpd.GeoDataFrame({"geometry":bou}, crs = edges.crs)

In [ ]:
# drop too-short linestrings
bou_red = bou[bou.length>length_threshold].copy()

# polygonize
poly = shapely.polygonize(
    np.array(
        bou_red.geometry
    )
)

gdf = gpd.GeoDataFrame(
    {
        "geometry": poly.geoms
    },
    crs = edges.crs
)

gdf["i"] = gdf.index

In [ ]:
gdf = gdf[[i!=[] for i in gdf.interiors]].reset_index(drop=True)

In [ ]:
gdf

***

# skeletonize separately

In [ ]:
delin = gpd.GeoDataFrame(
    {
        "geometry":
        [
            LineString(r) for r in gdf.interiors] + [LineString(highway.exterior)
        ]
    },
    crs = roads.crs
)
delin

In [ ]:
def unzip_line(geom, coordnum = 30):
    longline = [c for c in geom.coords]
    linestrings = []
    current_linestring = []
    for c in longline:
        current_linestring.append(c)
        if len(current_linestring) > coordnum:
            linestrings.append(LineString(current_linestring))
            del current_linestring
            current_linestring = [c]
    if current_linestring:
        linestrings.append(LineString(current_linestring))
    return linestrings

In [ ]:
all_lines = []

for geom in delin.geometry:
    all_lines+= (
        unzip_line(geom, my_coordnum)
    )

In [ ]:
lines = gpd.GeoDataFrame({"geometry": all_lines}, crs = roads.crs)
lines["i"] = lines.index
lines.explore(tiles="cartodb positron", column="i")

In [ ]:
skel = voronoi_skeleton(
    lines = lines.geometry,
    poly = highway,
    max_segment_length = my_maxseglen
)

In [ ]:
gdf.interiors

In [ ]:
gdf.filter([16,18,21], axis = 0).plot()